In [5]:
import os
import sqlite3
from sqlite3 import Error


def create_connection(db_file):
    conn = None
    try:
        conn = sqlite3.connect(db_file)
        conn.execute("PRAGMA foreign_keys = 1")
    except Error as e:
        print(e)

    return conn



In [6]:
def create_table(conn, create_table_sql):
    try:
        c = conn.cursor()
        c.execute(create_table_sql)
    except Error as e:
        print(e)

def insert_depts(conn, values):
    sql = ''' INSERT INTO Departments(DepartmentName)
              VALUES(?) '''
    cur = conn.cursor()
    cur.execute(sql, values)
    return cur.lastrowid


def insert_student(conn, values):
    sql = ''' INSERT INTO Students(StudentName, DepartmentId, DateOfBirth)
              VALUES(?,?,?) '''
    # ('Michael', 1, '1998-10-12'),
    cur = conn.cursor()
    cur.execute(sql, values)
    return cur.lastrowid


def select_all_students(conn):
    cur = conn.cursor()
    cur.execute("""SELECT * FROM Students INNER JOIN Departments USING(DepartmentId);""")

    rows = cur.fetchall()

    for row in rows:
        print(row)

    return rows

In [7]:
db_file = 'sample_data_py.db'
if os.path.exists(db_file):
    os.remove(db_file)

create_table_departments_sql = """ CREATE TABLE [Departments] (
    [DepartmentId] INTEGER  NOT NULL PRIMARY KEY,
    [DepartmentName] TEXT  NULL
); """

create_table_students_sql = """ CREATE TABLE [Students] (
    [StudentId] INTEGER  PRIMARY KEY NOT NULL,
    [StudentName] TEXT NOT NULL,
    [DepartmentId] INTEGER  NULL,
    [DateOfBirth] TEXT NULL,
    FOREIGN KEY(DepartmentId) REFERENCES Departments(DepartmentId)
);  """

conn = create_connection(db_file)


depts = ('IT', 'Physics', 'Arts', 'Math')
students = (
    ('Michael', 1, '1998-10-12'),
    ('John', 1, '1998-10-12'),
    ('Jack', 1, '1998-10-12'),
    ('Sara', 2, '1998-10-12'),
    ('Sally', 2, '1998-10-12'),
    ('Jena', None, '1998-10-12'),
    ('Nancy', 2, '1998-10-12'),
    ('Adam', 3, '1998-10-12'),
    ('Stevens', 3, '1998-10-12'),
    ('George', None, '1998-10-12')
)


In [8]:
with conn:
    create_table(conn, create_table_departments_sql)
    create_table(conn, create_table_students_sql)
    for values in depts:
        insert_depts(conn, (values, ))
    for values in students:
        insert_student(conn, values)
    select_all_students(conn)

    


(1, 'Michael', 1, '1998-10-12', 'IT')
(2, 'John', 1, '1998-10-12', 'IT')
(3, 'Jack', 1, '1998-10-12', 'IT')
(4, 'Sara', 2, '1998-10-12', 'Physics')
(5, 'Sally', 2, '1998-10-12', 'Physics')
(7, 'Nancy', 2, '1998-10-12', 'Physics')
(8, 'Adam', 3, '1998-10-12', 'Arts')
(9, 'Stevens', 3, '1998-10-12', 'Arts')


In [12]:
conn = create_connection(db_file)
import pandas as pd
sql_statement = "SELECT StudentName, DateOfBirth, DepartmentName FROM Students INNER JOIN Departments USING(DepartmentId);"
df = pd.read_sql_query(sql_statement, conn)
display(df)

,StudentName,DateOfBirth,DepartmentName
0,Michael,1998-10-12,IT
1,John,1998-10-12,IT
2,Jack,1998-10-12,IT
3,Sara,1998-10-12,Physics
4,Sally,1998-10-12,Physics
5,Nancy,1998-10-12,Physics
6,Adam,1998-10-12,Arts
7,Stevens,1998-10-12,Arts


In [16]:
sql_statement = """SELECT Departmentid from departments where departmentname=='Math';"""
df = pd.read_sql_query(sql_statement, conn)
display(df)

,DepartmentId
0,4


In [ ]:
""